# Parallelization in python scientific computing

In [2]:
import multiprocessing as mp

- In Linux system: it forks the sub processes, meaning each sub process will inheret the current state of parent program, they do have a 4GB argument restriction, but other than that, it is pretty flexible since no much pickling process got involved. The inherent parental variable is "copy-on-modify"

- In other system, it spawns sub processes, so a new interpreter gets launched, quite a lot pickling process involved, so like file IO, generating figures can not be parallelized, better to have a clear input and output in the enqueue and dequeue.

In [3]:
def func(x):
    return x+1

In [20]:
querys = [1,2,3,4]

1. pool.map will take the iteratble, make them a list, divide list to chunk and finally picklize the chunk to each sub-process (worker).
2. it runs out of order, but output will preserve the order, faster subprocess will have to wait slower one
3. result is just a list
4. result won't immediately return

In [5]:
pool = mp.Pool(processes=mp.cpu_count())
r = pool.map(func,querys)
print(r)

[2, 3, 4, 5]


1. pool.map_async will immediately return AsyncResult object and main program can proceed, but can not actually call AsyncResult.get() until all the subprocesses finish, so again, it preserve the order
2. you can mannually call r.wait() or pool.close() + pool.join() to make sure main program won't go until all workers finish. r is the AsyncResult obejct, but pool is the Pool object, Pool.close() instruct Pool to not take any new jobs, Pool.join() to instruct Pool to wait until all subprocesses have finished

In [6]:
pool = mp.Pool(processes=mp.cpu_count())
r = pool.map_async(func,querys)
print(r.get())

[2, 3, 4, 5]


1. pool.imap will not make them a list and divide to chunk, it will just take one from the queue, send to subprocess, and one after the another
2. it will preserve the order as well

In [9]:
pool = mp.Pool(processes=mp.cpu_count())
r = pool.imap(func,querys)
for item in r:
    print(item)

2
3
4
5


1. pool.imap_unordered will not preserve the order, return the result immediately, and you can access it, main program will continue

In [12]:
pool = mp.Pool(processes=mp.cpu_count())
r = pool.imap_unordered(func,querys)
for item in r:
    print(item)

2
3
4
5


1. pool.apply just can take additional argument
2. pool.apply_async, the same as pool.map_async

In [14]:
def func_apply(x,y):
    return x+y

In [15]:
y = 5
pool = mp.Pool(processes=mp.cpu_count())
r = [pool.apply(func_apply,args=(x,y)) for x in querys]
print(r)

[6, 7, 8, 9]


In [19]:
y = 5
pool = mp.Pool(processes=mp.cpu_count())
r = [pool.apply_async(func_apply,args=(x,y)) for x in querys]
for item in r:
    print(item.get())

6
7
8
9
